# Hepstat_Tutorial_Histfactory_Hists
Histfactory example. 

 with following objectives:
 * Create a workspace using histograms
 * Example operations at the workspace level




**Author:** Lailin XU  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Saturday, April 03, 2021 at 05:28 PM.</small></i>

In [1]:
import os

Import the ROOT libraries

In [2]:
import ROOT as R
from math import pow, sqrt
R.gROOT.SetStyle("ATLAS")

Welcome to JupyROOT 6.22/07


Prepare input files
=======================

In [3]:
inputhist = "data/h4l_toy_hists.root"
if not os.path.isfile(inputhist):
  print("Error! No input files found: {}".format(inputhist)) 
  pyhist = "hepstat_tutorial_genhists.py"
  if os.path.isfile(pyhist):
    cmd = "python3 {}".format(pyhist)
    os.system(cmd)

Signal mass point

In [4]:
mass = 125

Create a workspace
=======================

Create a Histfactory Measurement
-----------------------

First we set the Parameter of interest, and several constant parameters.

In [5]:
meas = R.RooStats.HistFactory.Measurement("meas", "meas")
meas.SetPOI("mu")


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Set the luminosity constant with a dummy uncertainty of 2%

In [6]:
meas.SetLumi( 1.0 )
meas.SetLumiRelErr( 0.02 )
meas.AddConstantParam("Lumi")

Create a channel and set the measured value of data

In [7]:
chan = R.RooStats.HistFactory.Channel( "SR" )
chan.SetStatErrorConfig(0.05, "Poisson")
chan.SetData("obsData", inputhist)

Create the signal sample and set its histogram

In [8]:
signal = R.RooStats.HistFactory.Sample( "signal", "sig_{}".format(mass), inputhist)

Add the parmaeter of interest and a systematic and try to make intelligent choice of upper bound

In [9]:
signal.AddNormFactor( "mu", 1, 0, 3)

Assign a dummy signal normalisation uncertainty (up, down with respect to the nominal)

In [10]:
signal.AddOverallSys( "signal_norm_uncertainty", 0.95, 1.05)

Add the signal sample to the Channel

In [11]:
chan.AddSample( signal )

Create the background sample and set its histogram

In [12]:
background = R.RooStats.HistFactory.Sample( "background", "bkg", inputhist )

Add bkg systematics

In [13]:
background.AddHistoSys("background_shape", "bkg_up", inputhist, "", "bkg_dn", inputhist, "")

Add the bkg sample to the Channel

In [14]:
chan.AddSample( background )

Add the Channel to the Meas

In [15]:
meas.AddChannel(chan)

Collect the histograms from their files, print some output, 

In [16]:
meas.PrintTree()

Measurement Name: meas	 OutputFilePrefix: 	 POI: mu	 Lumi: 1	 LumiRelErr: 0.02	 BinLow: 0	 BinHigh: 1	 ExportOnly: 0
Constant Params:  Lumi
Channels:
	 Channel Name: SR	 InputFile: 
	 Data:
	 	 InputFile: data/h4l_toy_hists.root	 HistoName: obsData	 HistoPath: 	 HistoAddress: 0x0
	 statErrorConfig:
	 	 RelErrorThreshold: 0.05	 ConstraintType: Poisson
	 Samples: 
	 	 Name: signal	 	 Channel: SR	 NormalizeByTheory: True	 StatErrorActivate: False
	 	 	 	 	 InputFile: data/h4l_toy_hists.root	 HistName: sig_125	 HistoPath: 	 HistoAddress: 0x0
	 	 Name: background	 	 Channel: SR	 NormalizeByTheory: True	 StatErrorActivate: False
	 	 	 	 	 InputFile: data/h4l_toy_hists.root	 HistName: bkg	 HistoPath: 	 HistoAddress: 0x0
	 End of Channel SR
[#2] INFO:HistFactory -- End Measurement: meas


Make workspace!
-----------------------

In [17]:
hist2workspace = R.RooStats.HistFactory.HistoToWorkspaceFactoryFast(meas)
ws = hist2workspace.MakeSingleChannelModel(meas, chan)

ws.SetName("myws")
ws.writeToFile("test_hf_ws_{}.root".format(mass))

False

[#2] PROGRESS:HistFactory -- Getting histogram data/h4l_toy_hists.root:/obsData
[#2] INFO:HistFactory -- Opened input file: data/h4l_toy_hists.root: 
[#2] PROGRESS:HistFactory -- Getting histogram data/h4l_toy_hists.root:/sig_125
[#2] PROGRESS:HistFactory -- Getting histogram data/h4l_toy_hists.root:/bkg
[#2] PROGRESS:HistFactory -- Getting histogram data/h4l_toy_hists.root:/bkg_up
[#2] PROGRESS:HistFactory -- Getting histogram data/h4l_toy_hists.root:/bkg_dn
[#2] PROGRESS:HistFactory -- 
-----------------------------------------
	Starting to process 'SR' channel with 1 observables
-----------------------------------------

[#2] INFO:HistFactory -- lumi str = [1,0,10]
[#2] INFO:HistFactory -- lumi Error str = nominalLumi[1,0,1.2],0.02
[#1] INFO:ObjectHandling -- RooWorkspace::import(SR) importing RooStats::HistFactory::FlexibleInterpVar::signal_SR_epsilon
[#2] INFO:HistFactory -- making normFactor: mu
[#2] INFO:HistFactory -- signal_SR has no variation histograms 
[#2] INFO:HistFactory

Draw all canvases 

In [18]:
from ROOT import gROOT 
gROOT.GetListOfCanvases().Draw()